In [1]:
import sys
sys.path.insert(0, '..')
from app.SIWeaviateClient import SIWeaviateClient
from app.schemas import Document, ImageChunk, ImageMetadata, TextChunk, TextMetadata
from app.processing.image.SIImageDescription import SIImageDescription
from app.processing.text.SIITranslator import SITranslator
from app.processing.BaseDocumentProcessor import BaseDocumentProcessor
import os
import fitz
import io
from PIL import Image
from unstructured.partition.pdf import partition_pdf
import re

image_descriptor = SIImageDescription()
translator = SITranslator()



/home/erwan/anaconda3/envs/scienceinfuse/lib/python3.9/site-packages/timm/models/layers/__init__.py:48: DeprecationWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", DeprecationWarning)


In [3]:
from app.processing.PDFProcessor import PDFProcessor
import os
import glob

def get_all_pdfs():
    pdf_files = []
    for root, dirs, files in os.walk("./data"):
        for file in files:
            if file.endswith(".pdf"):
                pdf_files.append(os.path.join(root, file))
    return pdf_files


with SIWeaviateClient() as client:
    pdf_paths = get_all_pdfs()
    for pdf_path in pdf_paths:
        print("==========================================")
        print(pdf_path)
        pdf = PDFProcessor(client, image_descriptor, translator, pdf_path)



./data/UniversScience/dossiers-pédagogiques/marc/cadran_solaire.pdf


/home/erwan/.cache/huggingface/modules/transformers_modules/microsoft/Florence-2-large-ft/c669c6b8bfbd7f0193fcb31f997879045a3612f3/processing_florence2.py:499: DeprecationWarning: invalid escape sequence \d
  """


ValueError: Unsupported number of image dimensions: 2

In [ ]:
combined = images + texts

# Sort the combined list
sorted_list = sorted(combined, key=lambda x: (x['page_number'], x['y_coordinate']))